# Managing passwords used by your Python Programs

**Austin Godber**  
**@godber**  
DesertPy Co-Organizer  
Meetup, Github and http://desertpy.com

DesertPy - 01/25/2017

# Whats the problem?

Say we have an HTTP server that requires a username and password to login to it, in this case HTTP Basic Authentication.

`python ./SimpleAuthServer.py 8080 desertpy:rocks`


Show that this pops up the username/password dialog in the browser

# Let's write some python to access that

In [1]:
import requests
from requests.auth import HTTPBasicAuth
r = requests.get('http://localhost:8080/file.txt',
                 auth=HTTPBasicAuth('desertpy', 'rocks'))
r.text

'This is a test file.\n'

## Github

Now lets just commit this code ... 

... wait, I just pushed passwords to a public github repo!

# Possible Solutions

* Pass in as Argument
* Pass in as Environment Variable
* Use `~/.netrc`
* Use fancy shared secret vault
* Use local keystore

# Pass in as argument


`python ./authget.py desertpy rocks`

Password shows up:
* in process listing to other users
* in bash history

# Pass in as environment variables

```bash
export USER=desertpy
export PASS=rocks
python ./authget.py
```

Password shows up:
* in bash history
* it sits around in shell memory

# Use your `~/.netrc` file

Put the following in `~/.netrc`

```
machine localhost
login desertpy
password rocks
```

Don't forget to `chmod 0600 ~/.netrc`

In [2]:
import requests
from requests.auth import HTTPBasicAuth
# note the omission of the auth kwarg
r = requests.get('http://localhost:8080/file.txt')
r.text

'This is a test file.\n'

Drawback:
* Password sits in cleartext file.

# Use a fancy shared secret vault

* https://www.vaultproject.io/
  * Super fancy
  * API for storing and retrieving passwords
  * Encrypted backend
  * "Short" lived tokens for authentication

What do we do with the token?  Pass it in and renew it.  Needs bootstrapping by person.  This is the cost of getting fancy.

# Use a local keystore

Install the python `keyring` module.

In [3]:
!pip install keyring

## Check out the keyring command

In [4]:
!keyring

Usage: keyring [get|set|del] SERVICE USERNAME

Options:
  -h, --help            show this help message and exit
  -p KEYRING_PATH, --keyring-path=KEYRING_PATH
                        Path to the keyring backend
  -b KEYRING_BACKEND, --keyring-backend=KEYRING_BACKEND
                        Name of the keyring backend


## Set password for service on command line

```
$ keyring set testhttpauth desertpy
Password for 'desertpy' in 'testhttpauth':
```

## Check that you can get the password on command line

```
$ keyring get testhttpauth desertpy
rocks
```

## Now use the keyring API

In [5]:
import requests
from requests.auth import HTTPBasicAuth
import keyring

user = "desertpy"
password = keyring.get_password('testhttpauth', user)
r = requests.get('http://localhost:8080/file.txt',
                 auth=HTTPBasicAuth(user, password))
r.text

'This is a test file.\n'